#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load huggingface embedding model (``model_name="sentence-transformers/all-mpnet-base-v2"``)
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)

**Useful links:**

- [Langchain Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)


In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

# ADD HERE YOUR CODE
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

c:\Users\sschn\OneDrive\Documents\Studium\3.Semester\Technische Informatik\Chatbot\TeamNovaBot\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\sschn\OneDrive\Documents\Studium\3.Semester\Technische Informatik\Chatbot\TeamNovaBot\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
text = "This is a test document."

# ADD HERE YOUR CODE
# Perform vector search
query_vector = embedding_model.embed_query(text)

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

Embedding vector length: 768
[-0.048951826989650726, -0.039862070232629776, -0.021562794223427773, 0.009908574633300304, -0.03810390084981918, 0.012684382498264313, 0.043494462966918945, 0.07183389365673065, 0.00974861066788435, -0.006987075321376324]


#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load pdf document "AI_Book.pdf" via langchain document loader: ``PyPDFLoader``
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

**Useful links:**

- [Langchain PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)
- [Langchain RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
# ADD HERE YOUR CODE
loader = PyPDFLoader(file_path=pdf_doc)

# Load and split documents in chunks
# ADD HERE YOUR CODE
pages_chunked = RecursiveCharacterTextSplitter(chunk_size =10000,chunk_overlap =20).split_documents(documents=loader.load())

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_text(text):
    # Remove surrogate pairs
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text
def clean_and_create_document(chunk):
    cleaned_text= clean_text(chunk.page_content)
    return Document(page_content=cleaned_text,metadata=chunk.metadata)

pages_chunked_cleaned = [clean_and_create_document (chunk) for chunk in pages_chunked]

print(pages_chunked[0])

page_content='Aurélien GéronHands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent SystemsSECOND EDITION
Boston Farnham Sebastopol Tokyo Beijing Boston Farnham Sebastopol Tokyo Beijing' metadata={'source': './AI_Book.pdf', 'page': 2}


In [4]:
print(pages_chunked[1])

page_content='978-1-492-03264-9
[LSI]Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurélien Géron
Copyright © 2019 Aurélien Géron. All rights reserved.
Printed in the United States of America.
Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles ( http://oreilly.com ). For more information, contact our corporate/institutional
sales department: 800-998-9938 or corporate@oreilly.com .
Editor:  Nicole Tache
Interior Designer:  David FutatoCover Designer:  Karen Montgomery
Illustrator:  Rebecca Demarest
June 2019:  Second Edition
Revision History for the Early Release
2018-11-05: First Release
2019-01-24: Second Release
2019-03-07: Third Release
2019-03-29: Fourth Release
2019-04-22: Fifth Release
See http://oreilly.com/catalog/errata.csp?isbn=9781492032649  for release details.
The O’Reilly logo 

In [5]:
print(f"Number of text chunks: {len(pages_chunked)}")

Number of text chunks: 501


#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain chroma db client
- store text document chunks and vector embeddings to vector databases

**Useful links:**

- [Langchain How To](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#initialization-from-client)

In [6]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# ADD HERE YOUR CODE
# Create a collection
collection = client.get_or_create_collection("collection_name")

# ADD HERE YOUR CODE
# Create chromadb
vector_db_from_client = Chroma(client=client,
                               collection_name="collection_name",
                               embedding_function=embedding_model,
                               )

In [7]:
from uuid import uuid4

#uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned[:50]))]

# ADD HERE YOUR CODE
#vector_db_from_client.add_texts(pages_chunked_cleaned,id=uuids)

uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned[:50]))]

# ADD HERE YOUR CODE
vector_db_from_client.add_documents(documents=pages_chunked_cleaned[:50],id=uuids)

['81b8ff79-c6d2-4eb0-b025-addc4f388720',
 'b11cd21b-6a30-4728-91ed-2c7679c9e3ba',
 '8f13a358-9387-487e-8d81-967688883f49',
 '93102210-af94-43cc-a115-d62b7c71de59',
 'cdf51df5-7238-4a19-8f31-5db6344d0fc6',
 '39507eaa-2c32-40d4-a595-d3d3c51c3b18',
 '9d63c713-c254-45ab-b6fe-d22db947ec1c',
 '49447fc5-b611-4f0b-bb8b-f03a4d83dbac',
 '933f56a3-b9ae-42a8-bc27-5a4582a95b3c',
 '01faa7fc-adb8-4abd-b8be-c5451e2c62ec',
 '24e81a42-65ab-4780-9247-47fbef4e9334',
 'cef7c1a9-3fe2-4b70-b22d-95a28e16335b',
 '5b92c19e-69d0-4c18-b578-5e2bb95812de',
 '53ca447b-1e98-4b2b-9bae-470faf51739a',
 '15097b21-ac72-4b9b-be52-1f015077c06f',
 '72378bcb-0f35-49c6-be02-fc8e64990d37',
 'b3b13ed1-ae08-4b71-a437-3a237dd9fa75',
 '678f9146-f831-4d54-b1fb-88e21765c3ce',
 'ecd82321-a4dd-4500-be3f-095073ffcfdf',
 'c8ca8a7f-6048-4e52-9216-698390f12441',
 'c94b8893-475e-4bd0-98ee-3a25e0833b58',
 '95a4999e-4478-4fb7-bda8-755d58671e16',
 '58584f76-7a0e-412a-aa4e-3e1f8bfde20b',
 '65049766-e30b-4fa8-aef7-7293eb1c32d0',
 '2872b0d9-60ca-

In [8]:
client.count_collections()

2

In [9]:
# client.delete_collection("ai_model_book")

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents


**Useful links:**

- [Langchain Query ChromaDB](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#query-directly)

In [10]:
search_query="Types of Machine Learning Systems"

results = vector_db_from_client.similarity_search(query=search_query,k=3)

for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n")

Types of Machine Learning Systems
There are so many different types of Machine Learning systems that it is useful to
classify them in broad categories based on:
Whether or not they are trained with human supervision (supervised, unsuper
vised, semisupervised, and Reinforcement Learning)
Whether or not they can learn incrementally on the fly (online versus batch
learning)
Whether they work by simply comparing new data points to known data points,
or instead detect patterns in the training data and build a predictive model, much
like scientists do (instance-based versus model-based learning)
These criteria are not exclusive; you can combine them in any way you like. For
example, a state-of-the-art spam filter may learn on the fly using a deep neural net
work model trained using examples of spam and ham; this makes it an online, model-
based, supervised learning system.
Lets look at each of these criteria a bit more closely.
Supervised/Unsupervised Learning
Machine Learning systems can be